# **PROCESAMIENTO DE LENGUAJE NATURAL**

## **Ejercicio: TF-IDF**

* En el siguiente ejercicio leeremos un archivo .bz2 el cual contiene en su interior un archivo con artículos de la página web https://www.biobiochile.cl en formato JSON

* Este archivo se encuentra en la carpeta "data" (./data/biobio_clean.bz2). (Para trabajar con el debes subirlo a tu espacio de trabajo en Colab).

In [4]:
import pandas as pd
import bz2

# Abre el archivo bz2
with bz2.open('./biobio_clean.bz2', "rt") as f:
    data = f.read()

# Convierte la cadena de datos en un DataFrame de pandas
dataset = pd.read_json(data)
dataset_r = dataset.copy(deep=True)
dataset.shape

/tmp/ipython-input-1511263097.py:9: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dataset = pd.read_json(data)


(26413, 10)

In [5]:
dataset.head()

,author,author_link,title,link,category,subcategory,content,tags,embedded_links,publication_datetime
0,Yerko Roa,/lista/autores/yroa,Colapsa otro segmento de casa que se derrumbó ...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-valparaiso,Noticia en Desarrollo Estamos recopilando m...,[],[],1565778000000
1,Valentina González,/lista/autores/vgonzalez,Policía busca a mujer acusada de matar a su pa...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Detectives de la Policía de Investigaciones ...,"[#parricidio, #PDI, #Pudahuel, #Región Metropo...",[https://media.biobiochile.cl/wp-content/uploa...,1565771820000
2,Felipe Delgado,/lista/autores/fdelgado,Dos detenidos en Liceo de Aplicación: protagon...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Dos detenidos fue el saldo de una serie de i...,"[#Incendio, #Liceo de Aplicación, #Región Metr...",[],1565772480000
3,Matías Vega,/lista/autores/mvega,Apoyo transversal: Senado aprueba en general p...,https://www.biobiochile.cl/noticias/nacional/c...,nacional,chile,La sala del Senado aprobó en general el proy...,"[#Inmigración, #Inmigrantes, #Ley, #Migración,...",[https://media.biobiochile.cl/wp-content/uploa...,1565772720000
4,Valentina González,/lista/autores/vgonzalez,Evacuación espontánea en Instituto Nacional po...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,La mañana de este miércoles se produjo una e...,"[#Carabineros, #FFEE, #Gases Lacrimógenos, #In...",[],1565772960000


In [6]:
# creamos una nueva columna titulo y contenido.
content = dataset['title'] + '. ' + dataset['content']
# obtenemos las clases
subcategory = dataset.subcategory

# dejamos en el dataset solo contenido de la noticia y categoria
dataset = pd.DataFrame({'content': content, 'category': subcategory})

In [7]:
dataset

,content,category
0,Colapsa otro segmento de casa que se derrumbó ...,region-de-valparaiso
1,Policía busca a mujer acusada de matar a su pa...,region-metropolitana
2,Dos detenidos en Liceo de Aplicación: protagon...,region-metropolitana
3,Apoyo transversal: Senado aprueba en general p...,chile
4,Evacuación espontánea en Instituto Nacional po...,region-metropolitana
...,...,...
26408,Naciones Unidas abre proceso de postulaciones ...,chile
26409,Fernando Astengo chocó en estado de ebriedad e...,region-metropolitana
26410,Detuvieron a hombre que arrojó combustible a u...,region-metropolitana
26411,Revelan identidad de 2 de 6 víctimas fatales e...,region-de-valparaiso


## **Ejercicios TF-IDF**


### **Ejercicio 01:**
**Implementar el algoritmo TF-IDF desde 0, no es permitido usar ninguna libreria de nlp, apenas se puede usar numpy, pandas u otra libreria de ciencia de datos**

In [8]:
import numpy as np
import pandas as pd
import math

# -------------------------
# Tokenización básica (split por espacio)
def tokenizar(doc):
    return doc.lower().split()

# -------------------------
# Construir vocabulario
def crear_vocabulario(docs):
    vocab = set()
    for doc in docs:
        vocab.update(doc)
    return list(vocab)

# -------------------------
# Calcular TF (frecuencia de término)
def calcular_tf(docs, vocab):
    tf = np.zeros((len(docs), len(vocab)))
    for i, doc in enumerate(docs):
        for palabra in doc:
            if palabra in vocab:
                j = vocab.index(palabra)
                tf[i, j] += 1
        # normalizar
        if len(doc) > 0:
            tf[i, :] = tf[i, :] / len(doc)
    return tf

# -------------------------
# Calcular IDF (inversa de frecuencia de documento)
def calcular_idf(docs, vocab):
    N = len(docs)
    idf = np.zeros(len(vocab))
    for j, palabra in enumerate(vocab):
        df = sum(1 for doc in docs if palabra in doc)
        idf[j] = math.log((N + 1) / (df + 1)) + 1
    return idf

# -------------------------
# Calcular matriz TF-IDF
def calcular_tfidf(docs, vocab):
    tf = calcular_tf(docs, vocab)
    idf = calcular_idf(docs, vocab)
    tfidf = tf * idf
    return tfidf

Probando con el dataset del ejecicio anterior.

In [9]:
docs_token = [tokenizar(texto) for texto in dataset["content"][:5]]  # primeros 5 docs
vocabulario = crear_vocabulario(docs_token)
tfidf = calcular_tfidf(docs_token, vocabulario)

print("Número de documentos:", len(docs_token))
print("Tamaño vocabulario:", len(vocabulario))

df_tfidf = pd.DataFrame(tfidf[:, :10], columns=vocabulario[:10])
print(df_tfidf)

Número de documentos: 5
Tamaño vocabulario: 481
   incluyeron    tórax.  educativa  valparaíso     forma       hoy  investiga  \
0     0.00000  0.000000   0.000000     0.02358  0.000000  0.000000   0.000000   
1     0.00000  0.006477   0.000000     0.00000  0.000000  0.000000   0.000000   
2     0.01849  0.000000   0.000000     0.00000  0.000000  0.009245   0.009245   
3     0.00000  0.000000   0.000000     0.00000  0.010818  0.000000   0.000000   
4     0.00000  0.000000   0.012201     0.00000  0.000000  0.000000   0.000000   

    periodo    lagos,  especiales,  
0  0.000000  0.000000     0.000000  
1  0.000000  0.006477     0.000000  
2  0.000000  0.000000     0.000000  
3  0.010818  0.000000     0.000000  
4  0.000000  0.000000     0.012201  


### **Ejercicio 02:**
**Compare sus resultados con los encontrados por las libreria nltk y scikit**